## Nacka kommun

* [#97](https://github.com/salgo60/ProjectOutdoorGyms/issues/97) 

* Notebook [97_outdoorgym](https://github.com/salgo60/ProjectOutdoorGyms/tree/main/Jupyter/97_outdoorgym.ipynb)

- [utegym_report_Nacka_2025_12_12](https://salgo60.github.io/ProjectOutdoorGyms/Jupyter/utegym_report_Nacka_2025_12_12.html)
- [utegym_report_Nacka_WDfix_2025_12_12](https://salgo60.github.io/ProjectOutdoorGyms/Jupyter/utegym_report_Nacka_WDfix_2025_12_12.html) - efter att ha lagt till utegym i Wikidata

## TODO Nacka
a) fixa tydligt o ni har persistenta unika identifierare för era utegym
b) bilder med fri licens [CC-0](https://creativecommons.org/public-domain/cc0/)

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-12-12 21:23:12


In [2]:
# current dir 
import os
os.getcwd()

'/Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter'

In [3]:
import os
import requests
import zipfile
import geopandas as gpd

# -------------------------------------------------------
# 1. Ladda ned ZIP-filen från Nacka
# -------------------------------------------------------

url = "https://infobank.nacka.se/Ext/oppnadata/utegym.zip"
zip_path = "/Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter/Nackautegym.zip"


print("Laddar ner utegym.zip ...")
r = requests.get(url)
with open(zip_path, "wb") as f:
    f.write(r.content)

print("Nedladdning klar:", zip_path)

# -------------------------------------------------------
# 2. Packa upp ZIP-filen
# -------------------------------------------------------

extract_dir = "/Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter/utegymNacka"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)

print("Uppackat till:", extract_dir)
print("Innehåll:", os.listdir(extract_dir))

# -------------------------------------------------------
# 3. Identifiera vilka GIS-format som finns
# -------------------------------------------------------

gpkg_file = None
shp_file = None

for root, dirs, files in os.walk(extract_dir):
    for f in files:
        if f.lower().endswith(".gpkg"):
            gpkg_file = os.path.join(root, f)
        if f.lower().endswith(".shp"):
            shp_file = os.path.join(root, f)

print("Hittad GeoPackage:", gpkg_file)
print("Hittad Shapefile:", shp_file)

# -------------------------------------------------------
# 4. Läs in geodata i GeoPandas
# -------------------------------------------------------

if gpkg_file:
    print("Läser GeoPackage...")
    gdf_local = gpd.read_file(gpkg_file)
elif shp_file:
    print("Läser Shapefile...")
    gdf_local = gpd.read_file(shp_file)
else:
    raise FileNotFoundError("Varken GeoPackage eller Shapefile hittades i ZIP-filen.")

# Säkerställ WGS84
gdf_local = gdf_local.to_crs("EPSG:4326")

print("Klar! Antal rader:", len(gdf_local))
gdf_local.head()



Laddar ner utegym.zip ...
Nedladdning klar: /Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter/Nackautegym.zip
Uppackat till: /Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter/utegymNacka
Innehåll: ['Shape', 'Metadata.txt', 'Användarvillkor.txt', 'dxf', 'Friskrivning.txt', 'Geopackage']
Hittad GeoPackage: /Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter/utegymNacka/Geopackage/utegym.gpkg
Hittad Shapefile: /Users/salgo/Documents/GitHub/ProjectOutdoorGyms/Jupyter/utegymNacka/Shape/utegym.shp
Läser GeoPackage...
Klar! Antal rader: 18


,namn,nummer,kommundel,ansvar_skotsel,pkid,ansvar,atgardstyp,typ,area,etikett,inlagd_den,andrad_den,lank,Renoverad,Longitud,Latitud,Adresspunkt,geometry
0,Nyckelviken,NaN,None,None,76489,None,None,utegym,475.908099,GYM,2022-11-01,2021-05-31,https://www.nacka.se/uppleva--gora/friluftsliv...,NaT,NaN,NaN,Nyckelviksvägen,POINT (18.17995 59.31783)
1,Velamsund,NaN,None,None,76491,None,None,utegym,625.345713,GYM,2018-11-01,2022-11-01,https://www.nacka.se/uppleva--gora/friluftsliv...,NaT,NaN,NaN,Velamsundsvägen 33,POINT (18.32001 59.34672)
2,Hellasgården,NaN,None,None,76602,Stockholm,None,utegym,725.504116,GYM,2018-11-06,2022-10-26,https://www.nacka.se/uppleva--gora/friluftsliv...,NaT,NaN,NaN,None,POINT (18.15909 59.28984)
3,Sickla strand,NaN,None,None,76448,None,None,utegym,52.616543,GYM,2018-11-01,2022-11-01,https://www.nacka.se/uppleva--gora/friluftsliv...,NaT,NaN,NaN,Sickla strand 25,POINT (18.1216 59.30231)
4,Hedvigslund,NaN,None,None,76422,None,None,utegym,77.625551,GYM,2018-11-01,2022-11-01,https://www.nacka.se/uppleva--gora/friluftsliv...,NaT,NaN,NaN,Anemonvägen 38,POINT (18.18702 59.24968)


In [4]:
import folium
from folium.features import DivIcon
# Beräkna medelpunkt
mean_lat = gdf_local.geometry.y.mean()
mean_lon = gdf_local.geometry.x.mean()

m = folium.Map(location=[mean_lat, mean_lon], zoom_start=12, tiles="OpenStreetMap",
    width=600,
    height=600)
# Lägg in varje utegym som marker
for idx, row in gdf_local.iterrows():
    lat = row.geometry.y
    lon = row.geometry.x

    name = None
    for col in ["name", "namn", "Namn", "NAME"]:
        if col in gdf_local.columns:
            name = row[col]
            break

    popup_text = f"{name}" if name else f"Utegym {idx}"

    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color="green", icon="dumbbell", prefix="fa")
    ).add_to(m)
# Rendera Folium-kartan till HTML (ren HTML-snutt, ingen separat fil)
map_html = m.get_root().render()


In [5]:
import pandas as pd
from datetime import datetime
import folium
from folium.features import DivIcon
import geopandas as gpd
from shapely.geometry import Point
from SPARQLWrapper import SPARQLWrapper, JSON
import numpy as np 
from folium.plugins import Fullscreen

# https://w.wiki/GcKm
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
query = """
SELECT DISTINCT ?entity ?entityLabel ?coord ?www WHERE {
  ?entity (wdt:P6104/(wdt:P361*)) wd:Q107186275;
    wdt:P17 wd:Q34;
    wdt:P131 wd:Q946647.
  OPTIONAL {?entity wdt:P625 ?coord}
  OPTIONAL {?entity wdt:P856 ?www}
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
} ORDER BY ?entityLabel
"""
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
res = sparql.query().convert()

wd_rows = []
for b in res["results"]["bindings"]:
    if "coord" not in b:
        continue
    qid = b["entity"]["value"].split("/")[-1]
    label = b.get("entityLabel", {}).get("value", "")
    wkt = b["coord"]["value"]

    try:
        lon, lat = wkt.replace("Point(", "").replace(")", "").split()
        lon, lat = float(lon), float(lat)
        #wd_rows.append({"qid": qid, "label": label, "lat": lat, "lon": lon}) 
        www = b.get("www", {}).get("value", None)

        wd_rows.append({
            "qid": qid,
            "label": label,
            "lat": lat,
            "lon": lon,
            "www": www
        })

    except:
        pass

df_wd = pd.DataFrame(wd_rows)
gdf_wd = gpd.GeoDataFrame(
    df_wd,
    geometry=gpd.points_from_xy(df_wd.lon, df_wd.lat),
    crs="EPSG:4326"
)

# För spatial join måste båda vara i metrisk CRS
gdf_local_m = gdf_local.to_crs(3857)
gdf_wd_m = gdf_wd.to_crs(3857)

joined = gpd.sjoin_nearest(
    gdf_local_m,
    gdf_wd_m[["qid","label","geometry"]],
    how="left",
    max_distance=50,
    distance_col="dist_m"
)

joined["qid"] = joined["qid"].astype("string")

# Avvikelser
joined["missing_in_wikidata"] = joined["qid"].isna()
joined["coord_mismatch"] = joined["dist_m"] > 25
joined["missing_wd_coord"] = False  # redan filtrerat

# Konvertera tillbaka till WGS84
joined = joined.to_crs(4326)

### ---- Skapa Wikidata-rapporttabell ----

# Slå ihop joined + Wikidata för enklare rapporttabell
wd_report = joined.merge(
    gdf_wd[['qid', 'label', 'lat', 'lon', 'www']],
    on="qid",
    how="right"
)

# Bygg statusfält
def status(row):
    # Wikidata-post som inte matchade någon Nacka-post
    if pd.isna(row["dist_m"]):
        return "Finns i Wikidata – saknas i Nacka"

    # Lokalt objekt som saknas i Wikidata (röd markör)
    if row.get("missing_in_wikidata") is True:
        return "Saknas i Wikidata"

    # Koordinatavvikelse
    if row.get("coord_mismatch") is True:
        return f"Koordinatavvikelse ({row['dist_m']:.1f} m)"

    return "Matchad"

wd_report["status"] = wd_report.apply(status, axis=1)

# Skapa klickbara länkar
wd_report["Q-länk"] = wd_report["qid"].apply(lambda q: f'<a href="https://www.wikidata.org/wiki/{q}" target="_blank">{q}</a>')
wd_report["Webb"] = wd_report["www"].apply(
    lambda u: f'<a href="{u}" target="_blank">{u}</a>' if isinstance(u, str) else ""
)

# Skapa tabell för HTML
wd_html_table = wd_report[[
    "label_y", "Q-länk", "lat", "lon", "status", "Webb"
]].to_html(escape=False, index=False)


In [6]:
wd_report

,namn,nummer,kommundel,ansvar_skotsel,pkid,ansvar,atgardstyp,typ,area,etikett,...,missing_in_wikidata,coord_mismatch,missing_wd_coord,label_y,lat,lon,www,status,Q-länk,Webb
0,Fisksätra IP,NaN,None,None,77006.0,idrott,None,utegym,130.056814,GYM,...,False,False,False,Fisksätra IP utegym,59.294590,18.253750,https://www.nacka.se/uppleva--gora/friluftsliv...,Matchad,"<a href=""https://www.wikidata.org/wiki/Q107400...","<a href=""https://www.nacka.se/uppleva--gora/fr..."
1,"Gröna dalen, Fisksätra",NaN,None,None,79873.0,None,None,utegym,505.436980,GYM,...,False,False,False,Gröna dalen utegym,59.290450,18.262519,https://www.nacka.se/uppleva--gora/friluftsliv...,Matchad,"<a href=""https://www.wikidata.org/wiki/Q137371...","<a href=""https://www.nacka.se/uppleva--gora/fr..."
2,Hedvigslund,NaN,None,None,76422.0,None,None,utegym,77.625551,GYM,...,False,False,False,Hedvigslund utegym,59.249640,18.186890,https://www.nacka.se/uppleva--gora/friluftsliv...,Matchad,"<a href=""https://www.wikidata.org/wiki/Q107401...","<a href=""https://www.nacka.se/uppleva--gora/fr..."
3,Hellasgården,NaN,None,None,76602.0,Stockholm,None,utegym,725.504116,GYM,...,False,False,False,Hellasgårdens utegym,59.289839,18.159092,https://motionera.stockholm/hitta-utegym/utegy...,Matchad,"<a href=""https://www.wikidata.org/wiki/Q107208...","<a href=""https://motionera.stockholm/hitta-ute..."
4,Henriksdalsberget,NaN,None,None,78164.0,None,None,utegym,565.465656,GYM,...,False,False,False,Henrikdalsberget utegym,59.310690,18.113590,https://www.nacka.se/uppleva--gora/friluftsliv...,Matchad,"<a href=""https://www.wikidata.org/wiki/Q107400...","<a href=""https://www.nacka.se/uppleva--gora/fr..."
5,Kocktorpssjön,NaN,None,None,77010.0,None,None,utegym,702.086429,GYM,...,False,False,False,Kocktorpssjöns utegym,59.312490,18.250340,https://www.nacka.se/uppleva--gora/friluftsliv...,Matchad,"<a href=""https://www.wikidata.org/wiki/Q107400...","<a href=""https://www.nacka.se/uppleva--gora/fr..."
6,Krokhöjden,NaN,None,None,76480.0,None,None,utegym,447.726559,GYM,...,False,False,False,Krokhöjdens utegym,59.329467,18.241071,https://www.nacka.se/uppleva--gora/friluftsliv...,Matchad,"<a href=""https://www.wikidata.org/wiki/Q107400...","<a href=""https://www.nacka.se/uppleva--gora/fr..."
7,Kvarnholmsparken,NaN,None,None,76603.0,None,None,utegym,213.157034,GYM,...,False,False,False,Kvarnholmen utegym,59.316955,18.144974,https://www.nacka.se/uppleva--gora/friluftsliv...,Matchad,"<a href=""https://www.wikidata.org/wiki/Q107400...","<a href=""https://www.nacka.se/uppleva--gora/fr..."
8,Långsjön,NaN,None,None,76577.0,None,None,utegym,485.488531,GYM,...,False,False,False,Långsjöns utegym,59.307330,18.193670,https://www.nacka.se/uppleva--gora/friluftsliv...,Matchad,"<a href=""https://www.wikidata.org/wiki/Q107400...","<a href=""https://www.nacka.se/uppleva--gora/fr..."
9,Nyckelviken,NaN,None,None,76489.0,None,None,utegym,475.908099,GYM,...,False,False,False,Nyckelviken utegym,59.317860,18.180020,https://www.nacka.se/uppleva--gora/friluftsliv...,Matchad,"<a href=""https://www.wikidata.org/wiki/Q107400...","<a href=""https://www.nacka.se/uppleva--gora/fr..."


In [7]:
from IPython.display import HTML
from folium.plugins import Fullscreen

mean_lat = gdf_local.geometry.y.mean()
mean_lon = gdf_local.geometry.x.mean()

m_local = folium.Map(location=[mean_lat, mean_lon], zoom_start=12, width=400, height=400)

for idx, row in gdf_local.iterrows():

    # Hantera ev. NaN och None
    def val(x):
        return x if pd.notna(x) and x not in ["None", None, "nan"] else None

    namn = val(row.get("namn"))
    pkid = val(row.get("pkid"))
    area = val(row.get("area"))
    ansvar_skotsel = val(row.get("ansvar_skotsel"))
    ansvar = val(row.get("ansvar"))
    kommundel = val(row.get("kommundel"))
    inlagd = val(row.get("inlagd_den"))
    andrad = val(row.get("andrad_den"))
    renoverad = val(row.get("Renoverad"))
    lank = val(row.get("lank"))
    adress = val(row.get("Adresspunkt"))

    # Formatera datum
    if isinstance(inlagd, pd.Timestamp):
        inlagd = inlagd.strftime("%Y-%m-%d")
    if isinstance(andrad, pd.Timestamp):
        andrad = andrad.strftime("%Y-%m-%d")
    if isinstance(renoverad, pd.Timestamp):
        renoverad = renoverad.strftime("%Y-%m-%d")

    # HTML-popup
    popup_html = f"""
    <div style='font-size:14px; line-height:1.4'>
        <b style='font-size:16px'>{namn or "Utegym"}</b><br><br>

        {'<b>PKID:</b> ' + str(pkid) + '<br>' if pkid else ''}
        {'<b>Kommundel:</b> ' + kommundel + '<br>' if kommundel else ''}
        {'<b>Typ:</b> ' + str(row.get("typ")) + '<br>' if val(row.get("typ")) else ''}
        {'<b>Area:</b> ' + str(area) + ' m²<br>' if area else ''}
        {'<b>Ansvar:</b> ' + ansvar + '<br>' if ansvar else ''}
        {'<b>Skötsel:</b> ' + ansvar_skotsel + '<br>' if ansvar_skotsel else ''}
        {'<b>Adress:</b> ' + adress + '<br>' if adress else ''}
        {'<b>Inlagd:</b> ' + inlagd + '<br>' if inlagd else ''}
        {'<b>Ändrad:</b> ' + andrad + '<br>' if andrad else ''}
        {'<b>Renoverad:</b> ' + renoverad + '<br>' if renoverad else ''}

        {'<b>Länk:</b> <a href="' + lank + '" target="_blank">' + lank + '</a><br>' if lank else ''}

        <br>
        <b>Koordinater:</b><br>
        Lat: {row.geometry.y:.6f}<br>
        Lon: {row.geometry.x:.6f}<br>
    </div>
    """

    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        icon=folium.Icon(color="green", icon="dumbbell", prefix="fa"),
        popup=folium.Popup(popup_html, max_width=350)
    ).add_to(m_local)

# Fullscreen-knapp
Fullscreen(position="topright").add_to(m_local)

map_local_html = m_local.get_root().render()


In [8]:
m_wd = folium.Map(
    location=[gdf_wd.lat.mean(), gdf_wd.lon.mean()],
    zoom_start=12,
    width=400,
    height=400
)

for idx, row in gdf_wd.iterrows():

    # Bygg popup HTML
    popup_html = f"""
    <div style='font-size:14px'>
        <b>{row['label']}</b><br>
        QID: <a href='https://www.wikidata.org/wiki/{row['qid']}' target='_blank'>{row['qid']}</a><br>
        Lat/Lon: {row['lat']:.5f}, {row['lon']:.5f}<br>
        { 'Webbplats: <a href="' + row["www"] + '" target="_blank">' + row["www"] + '</a><br>' 
            if ("www" in row and isinstance(row["www"], str)) else "" }
    </div>
    """

    folium.Marker(
        location=[row.lat, row.lon],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color="blue", icon="info-sign"),
    ).add_to(m_wd)

# Wikidata fullscreen-knapp
Fullscreen(position="topright").add_to(m_wd)

map_wd_html = m_wd.get_root().render()


In [9]:
from urllib.parse import quote
m_diff = folium.Map(location=[mean_lat, mean_lon], zoom_start=12, width=400, height=400)

for idx, row in joined.iterrows():

    color = None
    label = None

    # Vilken avvikelse?
    if row["missing_in_wikidata"]:
        color = "red"
        label = "Saknas i Wikidata"

    elif row["coord_mismatch"]:
        color = "orange"
        label = "Koordinatavvikelse (>25 m)"

    if color:

        # --- Hämta lokal metadata ---
        def val(x):
            return x if pd.notna(x) and x not in ["None", None, "nan"] else None

        namn = val(row.get("namn"))
        pkid = val(row.get("pkid"))
        adress = val(row.get("Adresspunkt"))
        lank = val(row.get("lank"))

        # Lokal koordinat
        lat_local = row.geometry.y
        lon_local = row.geometry.x

        # --- Wikidata info (om match finns) ---
        qid = row["qid"] if pd.notna(row["qid"]) else None

        if qid:
            wd_label = row["label"]
            # hämta WD-koord via original gdf
            wd_row = gdf_wd[gdf_wd["qid"] == qid]
            if len(wd_row):
                lat_wd = wd_row.iloc[0].lat
                lon_wd = wd_row.iloc[0].lon
            else:
                lat_wd = lon_wd = None

            dist = row["dist_m"]
        else:
            wd_label = None
            lat_wd = lon_wd = None
            dist = None

        # --- Bygg popup (HTML) ---
        popup_html = f"""
        <div style='font-size:14px; line-height:1.4'>
            <b style="font-size:16px; color:{'red' if color=='red' else 'orange'}">{label}</b><br><br>

            {'<b>Namn:</b> ' + namn + '<br>' if namn else ''}
            {'<b>PKID:</b> ' + str(pkid) + '<br>' if pkid else ''}
            {'<b>Adress:</b> ' + adress + '<br>' if adress else ''}
            {'<b>Länk:</b> <a href="' + lank + '" target="_blank">' + lank + '</a><br>' if lank else ''}

            <b>Lokala koordinater:</b><br>
            Lat: {lat_local:.6f}<br>
            Lon: {lon_local:.6f}<br><br>
        """

        # --- Om Wikidata saknas ---
        if not qid:
            # --- Bygg QuickStatements-block ---
            qs_lines = []
        
            qs_lines.append("CREATE")
            qs_lines.append(f'LAST|Len|"{namn}"')
            qs_lines.append("LAST|P31|Q107186275")  # outdoor gym
            qs_lines.append(f"LAST|P625|@{lat_local}/{lon_local}")
        
            if adress:
                qs_lines.append(f'LAST|P6375|"{adress}"')
            if lank:
                qs_lines.append(f'LAST|P856|"{lank}"')
        
            # Slå ihop till QS-text
            qs_text = "\n".join(qs_lines)
        
            # URL-enkoda för V2 (!)
            qs_url = "https://quickstatements.toolforge.org/#/v2=" + quote(qs_text, safe="")
        
            # Lägg till i popupen: länk + textblock
            popup_html += f"""
            <b style="color:red">Inget motsvarande Wikidataobjekt</b><br><br>
        
            <b>Skapa nytt objekt i Wikidata:</b><br>
            <a target="_blank" href="{qs_url}">Öppna QuickStatements V2 (förifyllt)</a>
            <br><br>
        
            <b>QuickStatements-kommando:</b><br>
            <pre style='background:#f0f0f0; padding:8px; border:1px solid #ccc; white-space:pre-wrap;'>
            {qs_text}
            </pre>
            </div>
            """
        else:
            # --- Om Wikidata finns ---
            popup_html += f"""
            <hr>
            <b>Wikidata-match:</b><br>
            Label: {wd_label}<br>
            QID: <a href="https://www.wikidata.org/wiki/{qid}" target="_blank">{qid}</a><br>
            {'Lat: ' + str(lat_wd) + '<br>' if lat_wd else ''}
            {'Lon: ' + str(lon_wd) + '<br>' if lon_wd else ''}

            {'<b>Avstånd lokal ↔ Wikidata:</b> ' + str(round(dist,1)) + ' m<br><br>' if dist else ''}

            <b>Öppna kartor:</b><br>
            <a target="_blank" href="https://www.google.com/maps?q={lat_local},{lon_local}">Google Maps (lokal)</a><br>
            <a target="_blank" href="https://www.openstreetmap.org/?mlat={lat_local}&mlon={lon_local}#map=17/{lat_local}/{lon_local}">
                OpenStreetMap (lokal)
            </a><br>
            <a target="_blank" href="https://www.google.com/maps?q={lat_wd},{lon_wd}">
                Google Maps (Wikidata)
            </a><br>
            <a target="_blank" 
               href="https://www.openstreetmap.org/?mlat={lat_wd}&mlon={lon_wd}#map=17/{lat_wd}/{lon_wd}">
               OpenStreetMap (Wikidata)
            </a><br>
            </div>
            """

        folium.Marker(
            location=[lat_local, lon_local],
            popup=folium.Popup(popup_html, max_width=350),
            icon=folium.Icon(color=color)
        ).add_to(m_diff)

# Fullscreen
Fullscreen(position="topright").add_to(m_diff)

map_diff_html = m_diff.get_root().render()


In [10]:
import pandas as pd
from datetime import datetime

date_tag = datetime.now().strftime("%Y_%m_%d")
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

output_path = f"./utegym_report_Nacka_WDfix_{date_tag}.html"

row_count = len(gdf_local)
crs = gdf_local.crs.to_string() if gdf_local.crs else "None"
columns = ", ".join(gdf_local.columns)

preview_html = gdf_local.head(20).to_html()

html_report = f"""
<html>
<head>
    <meta charset="UTF-8">
    <title>Utegymrapport with updated WD objects {date_tag}</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 40px;
        }}
        h1, h2 {{
            color: #333;
        }}
        a {{
            color: #1a0dab;
        }}
        .section {{
            margin-bottom: 30px;
        }}
        .map-row {{
            display: flex;
            flex-direction: row;
            gap: 40px;
        }}
        .map-box {{
            width: 500px;   /* karta + marginal */
            height: auto;
            margin-right: 20px;
     }}

}}

    </style>
</head>
<body>

<h1>Utegymrapport with updated WD objects</h1>
<p><strong>Issue:</strong> <a href="https://github.com/salgo60/ProjectOutdoorGyms/issues/97">#97</a></p>
<p><strong>Notebook:</strong> <a href="https://github.com/salgo60/ProjectOutdoorGyms/blob/main/Jupyter/97_outdoorgym.ipynb">97_outdoorgym.ipynb</a></p>
<p><strong>Report 1 before updating:</strong> <a href="https://salgo60.github.io/ProjectOutdoorGyms/Jupyter/utegym_report_Nacka_2025_12_12.html">utegym_report_Nacka_2025_12_12.html</a></p>


<p>Genererad: <strong>{timestamp}</strong></p>

<div class="section">
    <h2>Kartor (lokala + Wikidata)</h2>
    <p></p>
    <div class="map-row">
        <div class="map-box">
            <h3>Karta 1: Nacka</h3>
            {map_local_html}
        </div>
        <div class="map-box">
            <h3>Karta 2: Wikidata</h3>
            &nbsp;&nbsp;{map_wd_html}
         </div>
         <div class="map-box">
           <h3>Karta 3: Avvikelser</h2>
            &nbsp;&nbsp;{map_diff_html}
            </div>   
</div>
<p></p>
<div class="section">
    <h2>Datasetinformation Nacka kommun</h2>
    <p><strong>Antal rader:</strong> {row_count}</p>
    <p><strong>CRS:</strong> {crs}</p>
    <p><strong>Kolumner:</strong> {columns}</p>
</div>

<div class="section">
    <h2>Förhandsgranskning </h2>
    {preview_html}
</div>

<div class="section">
    <h2>Wikidata-poster i området</h2>
    <p>Denna tabell visar alla utegym som finns i Wikidata för Nacka, deras koordinater, status och länkar.</p>
    {wd_html_table}
</div>


</body>
</html>
"""

with open(output_path, "w", encoding="utf-8") as f:
    f.write(html_report)

output_path


'./utegym_report_Nacka_WDfix_2025_12_12.html'

In [11]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))


Date: 2025-12-12 21:23:14
Total time elapsed: 00 minutes 01.49 seconds
